In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py 

import petsc4py
petsc4py.init()
from petsc4py import PETSc

In [2]:
size = 64

In [3]:
grid_beta  = np.load("grid_beta.npy")
grid_delta = np.load("grid_delta.npy")

_beta  = np.zeros((4096,64),dtype=np.float64)
_delta = np.zeros((4096,64),dtype=np.float64)

In [4]:
for i in range(size):
    _beta[64*i:64*(i+1),:]  = grid_beta[:,:,i]
    _delta[64*i:64*(i+1),:] = grid_delta[:,:,i]

In [5]:
beta = PETSc.Mat().create(PETSc.COMM_WORLD)
beta.setSizes([size**2,size])
beta.setType('dense')
beta.setFromOptions()
beta.setUp()

In [6]:
Istart, Iend = beta.getOwnershipRange()
for I in np.arange(Istart,Iend):
    for J in range(size):
        beta[I,J] = _beta[I,J]
beta.assemblyBegin()
beta.assemblyEnd()

In [7]:
view_beta = PETSc.Viewer().createBinary('beta.dat', 'w')
view_beta(beta)

In [8]:
delta = PETSc.Mat().create(PETSc.COMM_WORLD)
delta.setSizes([size**2,size])
delta.setType('dense')
delta.setFromOptions()
delta.setUp()

In [9]:
Istart, Iend = delta.getOwnershipRange()
for I in np.arange(Istart,Iend):
    for J in range(size):
        delta[I,J] = _delta[I,J]
delta.assemblyBegin()
delta.assemblyEnd()

In [10]:
view_delta = PETSc.Viewer().createBinary('delta.dat', 'w')
view_delta(delta)

In [11]:
beta.destroy()
delta.destroy()